# 🧱 02 – Data Preparation & Cleaning

This notebook creates a clean, structured version of the 2019 flat racing dataset for modelling.

We apply:
- Safe renames and value corrections identified in the previous data integrity audit
- Minor derived columns (e.g. did_finish)
- Optional filters for clean modelling subsets

---

## ✅ Why This Step Matters

Rather than modifying raw data in-place, we follow **best practice** in data workflows:

- 📦 **Raw files remain untouched**, ensuring a reliable reference point and audit trail
- 🧪 **All fixes are applied in memory** to a copied DataFrame
- 💾 **Processed data is saved separately**, ready for use in modelling notebooks

This makes our pipeline:
- Reproducible
- Traceable
- Safe for collaboration and experimentation

---
By the end of this notebook, we’ll have:
- `horses_2019_clean.csv` – cleaned, horse-level data (one row per runner)
- `races_2019_clean.csv` – cleaned, race-level metadata (one row per race)

We’ll only merge them later if a model or analysis step requires both levels.




In [9]:
import pandas as pd
import numpy as np

# Set file paths
YEAR = 2019
horses_path = f"data/raw/horses_{YEAR}.csv"
races_path = f"data/raw/races_{YEAR}.csv"

# Load
horses = pd.read_csv(horses_path)
races = pd.read_csv(races_path)

print("✅ Raw data loaded")


✅ Raw data loaded


---

## 📊 Pre-Cleaning Audit Snapshot

Before making any changes, we take a quick look at key columns to establish a baseline.

This lets us confirm that:
- Our fixes behave as expected
- No columns are accidentally dropped, renamed, or altered

We'll use this to validate each transformation in the steps ahead.


In [10]:
# Preview raw structure
print("🔢 Columns in horses_raw:", horses.columns.tolist())
print("🔍 Sample values:")
print(horses[['decimalPrice', 'position']].sample(5))

# Optional: count key placeholder values
print("⚠️ Horses with position == 40 (non-finishers placeholder):", (horses['position'] == 40).sum())
print("🎲 'decimalPrice' stats:")
print(horses['decimalPrice'].describe())


🔢 Columns in horses_raw: ['rid', 'horseName', 'age', 'saddle', 'decimalPrice', 'isFav', 'trainerName', 'jockeyName', 'position', 'positionL', 'dist', 'weightSt', 'weightLb', 'overWeight', 'outHandicap', 'headGear', 'RPR', 'TR', 'OR', 'father', 'mother', 'gfather', 'runners', 'margin', 'weight', 'res_win', 'res_place']
🔍 Sample values:
        decimalPrice  position
160292      0.019608        11
85252       0.010000        14
122870      0.114943         1
118465      0.066667         1
151957      0.045455         4
⚠️ Horses with position == 40 (non-finishers placeholder): 8033
🎲 'decimalPrice' stats:
count    171849.000000
mean          0.120026
std           0.118543
min           0.001767
25%           0.038462
50%           0.083333
75%           0.163934
max           0.961538
Name: decimalPrice, dtype: float64


---

## ✅ Pre-Cleaning Audit Complete

The raw data structure and key placeholder values were as expected.

- `decimalPrice` contains valid implied probabilities (0 < p < 1)
- `position == 40` appears 8,033 times and needs replacing
- Column structure matches the dataset documentation

We now begin applying our processing steps, one at a time.


### ✅ Fix 1 – Rename `decimalPrice` to `implied_prob`

In our audit, we confirmed that the `decimalPrice` column actually contains **implied win probabilities**, not raw decimal odds.

To prevent confusion and make its purpose clearer in downstream modelling, we rename the column to `implied_prob`.

This makes the data easier to work with and aligns with probability-based feature naming conventions.

In [12]:
# Rename mislabelled column
if 'decimalPrice' in horses.columns:
    horses = horses.rename(columns={'decimalPrice': 'implied_prob'})

✅ **Check: Confirm column rename worked**

We do a quick check to confirm that:
- The new `implied_prob` column exists
- It has numeric values in the expected range (0–1)
- The old `decimalPrice` column is no longer present

This helps ensure the rename was applied correctly before continuing.

In [13]:
print("implied_prob" in horses.columns)  # Should be True
horses['implied_prob'].describe().head()  # Sanity-check stats

True


count    171849.000000
mean          0.120026
std           0.118543
min           0.001767
25%           0.038462
Name: implied_prob, dtype: float64

### ✅ Fix 2 – Replace `position == 40` with `NaN`

In our audit, we saw that 8,033 entries in the `position` column had the value `40`.

This value acts as a placeholder for non-finishers and should not be treated as a true finishing place.

To avoid misleading our model, we replace these values with `NaN`.

We'll confirm the fix by:

- Checking that there are now **0 entries** with `position == 40`
- Verifying that the number of `NaN` values in `position` has increased accordingly



In [15]:
# Replace position == 40 with NaN to flag non-finishers
horses['position'] = horses['position'].replace(40, np.nan)

# Confirm no '40' values remain
print("✅ position == 40:", (horses['position'] == 40).sum())

# Confirm total NaNs (should now include the 8033 converted)
print("✅ Total NaN in position:", horses['position'].isna().sum())

✅ position == 40: 0
✅ Total NaN in position: 8033


✅ **Result: Placeholder removal successful**

- All `position == 40` values have been correctly replaced with `NaN`
- `NaN` now signals a non-finisher, without corrupting the numerical integrity of the `position` column
- This prepares the data for a clean finish flag and simplifies future modelling logic


### ✅ Fix 3 – Add `did_finish` Flag

The `position` column tells us where a horse finished — but only if they actually completed the race.

If a horse failed to finish (e.g. pulled up, unseated, refused), the `position` value is missing (`NaN`).

To make this clearer and easier to work with, we create a new binary column: `did_finish`.

- `1` means the horse finished the race
- `0` means the horse did not finish

This flag improves clarity, simplifies filtering, and allows us to explore finish reliability as a potential feature later.


In [19]:
# Create binary finish flag
horses['did_finish'] = horses['position'].notna().astype(int)

# Check value counts of the new column
print(horses['did_finish'].value_counts(dropna=False))

did_finish
1    163816
0      8033
Name: count, dtype: int64


✅ **Result: `did_finish` flag successfully added**

- 163,816 horses marked as finishers (`did_finish == 1`)
- 8,033 horses marked as non-finishers (`did_finish == 0`)
- This new feature makes it easy to include/exclude non-finishers or even model completion likelihood

The core fixes are now complete and the data is ready for optional filtering, enrichment, or merging.


## 💾 Save Cleaned Horse-Level Data

We now save the cleaned `horses` dataset to a processed file.

- The original raw file remains untouched
- No merge is performed here — we maintain table separation
- Merging will be done later, only when required for modelling or analysis
## 💾 Also Save Untouched Race Data for Consistency

Although we haven’t made any changes to the `races` dataset yet, we save a copy into our `processed/` folder.

This avoids confusion later and ensures all modelling notebooks refer to data in one place.

In [23]:
# Save cleaned horse-level data
horses.to_csv(f"data/processed/horses_{YEAR}_clean.csv", index=False)

# Save races data as-is for clarity and consistency
races.to_csv(f"data/processed/races_{YEAR}_clean.csv", index=False)

print("✅ Cleaned horse data saved (no merging)")
print("✅ Races data saved (untouched)")

✅ Cleaned horse data saved (no merging)
✅ Races data saved (untouched)


---

## ✅ Data Preparation Complete

This notebook applied all known and verified cleaning steps to the 2019 horse-level data.

We have:

- 🧼 Renamed mislabelled columns for clarity (`decimalPrice → implied_prob`)
- 🛠️ Handled placeholders (`position == 40` → `NaN`)
- 📍 Added a clear `did_finish` flag
- 💾 Saved a clean version of the dataset for downstream use
- 📂 Stored a copy of the untouched race-level data to maintain consistency

We now have a reliable, readable, and reproducible dataset ready for:

- 🧪 Feature engineering
- 🧠 Exploratory analysis
- 🏇 Targeted filtering (e.g. Derby-like races)
- 🧮 Modelling workflows

Next step: EDA or feature prep — depending on your focus.
